# Contrastive Language-Image Pretraining with SogCLR

### **Introduction**

In this tutorial, you will learn how to conduct contrastive language-image pretraining by optimizing the [Global Contrastive Loss](https://arxiv.org/abs/2202.12387) (GCL) on a subset of the [Conceptual Captions](https://ai.google.com/research/ConceptualCaptions/) dataset. Also, you will learn how to evaluate the model on retrieval task using the [MSCOCO](https://cocodataset.org/#home) dataset and zero-shot classification task using the [ImageNet](https://www.image-net.org/challenges/LSVRC/index.php) dataset. The code is based on [iSogCLR's](https://github.com/zhqiu/contrastive-learning-iSogCLR) codebase, which includes the implementation of CLIP, SogCLR and iSogCLR.

### Preparation

First, we:

1. Download the source code and data
2. Install required packages

In [1]:
!git clone -b project https://github.com/xywei00/csce689_iSogCLR.git iSogCLR

!export PYTHONPATH="$PYTHONPATH:./iSogCLR/bimodal_exps"
!export HUGGINGFACE_HUB_CACHE='./checkpoints/huggingface'
!mkdir checkpoints

!gdown 13wD5rqZtt9384ixjbMCaOqBajOUGMNbt    # clip.tar.gz
!gdown 1HvlkHpaKptO9I7WsAH_FJSLGek0rAK8-    # cc3m_subset_100k.tar.gz
!gdown 1MU0XnP01j4lCyWANRxDBgkgoPsyQdxFX    # ms_coco_val.tar.gz
!gdown 1SUK9F3ZBxdorGpsTS0QjO0gl9Bpbg0d-    # val.tar

!mkdir datasets
print(1)
!mkdir -p datasets/imagenet
print(2)
!tar xf clip_train.tar.gz
print(3)
!tar xf cc3m_subset_100k.tar.gz -C datasets
print(4)
!tar xf mscoco_val.tar.gz -C datasets
print(5)
!tar xf val.tar -C datasets/imagenet
print(6)

!pip install -r ./iSogCLR/requirements_colab.txt    # there may be pip warnings/ errors, should be fine to ignore them

Cloning into 'iSogCLR'...
remote: Enumerating objects: 314, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 314 (delta 26), reused 37 (delta 18), pack-reused 261 (from 1)
Receiving objects: 100% (314/314), 153.48 KiB | 4.15 MiB/s, done.
Resolving deltas: 100% (147/147), done.
Downloading...
From: https://drive.google.com/uc?id=13wD5rqZtt9384ixjbMCaOqBajOUGMNbt
To: /content/clip_train.tar.gz
100% 4.06M/4.06M [00:00<00:00, 113MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1HvlkHpaKptO9I7WsAH_FJSLGek0rAK8-
From (redirected): https://drive.google.com/uc?id=1HvlkHpaKptO9I7WsAH_FJSLGek0rAK8-&confirm=t&uuid=7ae2acaa-8f2b-4063-9a8e-d4e82eaf33b2
To: /content/cc3m_subset_100k.tar.gz
100% 3.07G/3.07G [00:27<00:00, 110MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1MU0XnP01j4lCyWANRxDBgkgoPsyQdxFX
From (redirected): https://drive.google.com/uc?id=1MU0XnP01j4lCyWANRxDBgkgoPsyQdxFX&confirm=

### Training

The following command runs the training script to train a ResNet50 (pretrained on ImageNet) and a DistilBERT (pretrained on BookCorpus and English Wikipedia) on the cc3m dataset using the SogCLR loss for 30 epochs with temperature 0.01.

## **clip + AdamW**

In [8]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/clip_and_adamw \
    --init_model \
    --use_amp \
    --ita_type clip \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --opt AdamW

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
Start training
Train Epoch: [0]  [  0/781]  eta: 2:58:28  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 12.0767  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 0.0000  grad_tau_text: 0.0000  b_I: 0.0000  b_T: 0.0000  v: 0.0000  lamda: 0.0000  weights_image_pos: 0.0000  weights_text_pos: 0.0000  time: 13.7108  data: 2.0538  max mem: 11691
Train Epoch: [0]  [ 50/781]  eta: 0:14:37  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 7.4091  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 0.0000  grad_tau_text: 0.0000  b_I: 0.0000  b_T: 0.0000  v: 0.0000  lamda: 0.0000  weights_image_pos: 0.0000  weights_text_pos: 0.0000  time: 1.0457  data: 0.3425  max mem: 11691
Train Epoch: [0]  [100/781]  eta: 0:12:51  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 6.1599  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image

### Evaluation

The following command runs the evaluation script to evaluate the retrieval performance of the trained model on the MSCOCO validation dataset and the zero-shot classification performance on the ImageNet validation dataset. The evaluation command is obtained by appending `--evaluate --checkpoint /path/to/your/checkpoint --zs_dataset imagenet --zs_datafolder /path/to/imagenet/val` to the training command.

In [9]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/clip_and_adamw \
    --init_model \
    --use_amp \
    --ita_type clip \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --evaluate --checkpoint './output/clip_and_adamw/checkpoint_30.pth' \
    --zs_dataset imagenet --zs_datafolder ./datasets/imagenet/val

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
load checkpoint from ./output/clip_and_adamw/checkpoint_30.pth
Start training
Computing features for evaluation...
Evaluation time 0:01:20
coco val: {'txt_r1': 12.22, 'txt_r5': 31.32, 'txt_r10': 43.22, 'txt_r_mean': 28.919999999999998, 'img_r1': 9.06873525530809, 'img_r5': 24.863049302251188, 'img_r10': 35.77112239593746, 'img_r_mean': 23.234302317832245, 'r_mean': 26.077151158916124}
zeroshot: {'zeroshot_top1': 21.364, 'zeroshot_top3': 33.84, 'zeroshot_top5': 39.474, 'zeroshot_top10': 47.436}
Training time 0:11:31


## **Benchmarks**

The following results are recall at 1 results on the provided MSCOCO and ImageNet datasets. The first row of results are from the model trained using the CLIP loss, and the second row of results are from the model trained using the SogCLR loss. All results are based on a batch size of 128 for 30-epoch pretraining. IR@1 denotes the recall at 1 of image retrieval on MSCOCO, TR@1 denotes the recall at 1 of text retrieval on MSCOCO, and ACC@1 denotes the top 1 accuracy on ImageNet. Average denotes the average of the three metrics.

| Method | MSCOCO TR@1 | MSCOCO IR@1 | ImageNet ACC@1 | Average |
|:----------:|:--------:|:--------:|:--------:|:--------:|
| CLIP | 12.0 | 9.32 | 21.35 | 14.22 |
| SogCLR |  14.38  |  10.73  | 24.54 | 16.55 |

## -----------------------------------------------------------------------------------------------------------------------------------------------------------

##  **Clip + Novograd**

In [26]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/clip_and_novograd \
    --init_model \
    --use_amp \
    --ita_type clip \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --opt NovoGrad

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
Start training
Train Epoch: [0]  [  0/781]  eta: 3:06:24  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 12.0786  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 0.0000  grad_tau_text: 0.0000  b_I: 0.0000  b_T: 0.0000  v: 0.0000  lamda: 0.0000  weights_image_pos: 0.0000  weights_text_pos: 0.0000  time: 14.3202  data: 2.3738  max mem: 11693
/content/iSogCLR/bimodal_exps/optim/novograd.py:76: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1642.)
  p.data.add_(-step_size, m)
Train Epoch: [0]  [ 50/781]  eta: 0:15:47  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 13.5325  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 0.0000  grad_tau_t

### **Evaluation**

In [33]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_cc3m_g0.8_e30 \
    --init_model \
    --use_amp \
    --ita_type clip \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 3 \
    --evaluate --checkpoint './output/clip_and_novograd/checkpoint_30.pth' \
    --zs_dataset imagenet --zs_datafolder ./datasets/imagenet/val

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
load checkpoint from ./output/clip_and_novograd/checkpoint_30.pth
Start training
Computing features for evaluation...
Evaluation time 0:01:17
coco val: {'txt_r1': 9.84, 'txt_r5': 27.22, 'txt_r10': 38.62, 'txt_r_mean': 25.22666666666667, 'img_r1': 6.889519772881763, 'img_r5': 20.89247870766524, 'img_r10': 31.200767723619496, 'img_r_mean': 19.6609220680555, 'r_mean': 22.443794367361086}
zeroshot: {'zeroshot_top1': 14.172, 'zeroshot_top3': 26.954, 'zeroshot_top5': 33.898, 'zeroshot_top10': 44.386}
Training time 0:11:23


## -----------------------------------------------------------------------------------------------------------------------------------------------------------

## **isogclr_new + AdamP**

In [2]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_new_and_AdamP \
    --init_model \
    --use_amp \
    --ita_type isogclr_new \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --opt AdamP

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
Start training
/content/iSogCLR/bimodal_exps/optim/adamp.py:84: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1642.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)
Train Epoch: [0]  [  0/781]  eta: 3:26:29  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 0.2126  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 4.6248  grad_tau_text: 4.1886  b_I: 5.0000  b_T: 3.2149  v: 0.0000  lamda: 0.0000  weights_image_pos: 0.0000  weights_text_pos: 0.0000  time: 15.8631  data: 2.1485  max mem: 11787
Train Epoch: [0]  [ 50/781]  eta: 0:16:14  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 0.0902  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 4.9862  

In [3]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_new_and_AdamP \
    --init_model \
    --use_amp \
    --ita_type isogclr_new \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --evaluate --checkpoint './output/isogclr_new_and_AdamP/checkpoint_30.pth' \
    --zs_dataset imagenet --zs_datafolder ./datasets/imagenet/val

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
load checkpoint from ./output/isogclr_new_and_AdamP/checkpoint_30.pth
Start training
Computing features for evaluation...
Evaluation time 0:01:19
coco val: {'txt_r1': 13.96, 'txt_r5': 33.7, 'txt_r10': 45.92, 'txt_r_mean': 31.19333333333334, 'img_r1': 10.584189691710984, 'img_r5': 27.318165460434244, 'img_r10': 37.68243432364349, 'img_r_mean': 25.194929825262903, 'r_mean': 28.194131579298123}
zeroshot: {'zeroshot_top1': 26.808, 'zeroshot_top3': 39.866, 'zeroshot_top5': 45.3, 'zeroshot_top10': 52.156}
Training time 0:11:37


## ----------------------------------------------------------------------

## isogclr_new + Radam

In [4]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_new_and_RAdam \
    --init_model \
    --use_amp \
    --ita_type isogclr_new \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --opt RAdam

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
Start training
/scratch/user/kiran3566/CSCE_636_Final_Project/iSogCLR/bimodal_exps/optim/radam.py:50: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value = 1) (Triggered internally at /opt/conda/conda-bld/pytorch_1729647329220/work/torch/csrc/utils/python_arg_parser.cpp:1642.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
Train Epoch: [0]  [  0/781]  eta: 1:29:10  lr: 0.000010  lr_temp_net: 0.00000100  loss_ita: 0.1583  avg_image_tau: 0.0100  avg_text_tau: 0.0100  cur_eta: 0.0000  grad_tau_image: 4.9776  grad_tau_text: 4.4716  b_I: 5.0000  b_T: 4.2031  v: 0.0000  lamda: 0.0000  weights_image_pos: 0.0000  weights_text_pos: 0.0000  time: 6.8505  data: 1.4823  max mem: 13434
Train Epoch: [0]  [ 50/781]  eta: 0:07:24  

### Validation

In [5]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/isogclr_new_and_RAdam \
    --init_model \
    --use_amp \
    --ita_type isogclr_new \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --evaluate --checkpoint './output/isogclr_new_and_RAdam/checkpoint_30.pth' \
    --zs_dataset imagenet --zs_datafolder ./datasets/imagenet/val

Creating retrieval dataset
len of train_dataset: 100000
len of coco val: 5000
Creating model
load checkpoint from ./output/isogclr_new_and_RAdam/checkpoint_30.pth
Start training
Computing features for evaluation...
Evaluation time 0:01:17
coco val: {'txt_r1': 13.66, 'txt_r5': 33.26, 'txt_r10': 45.24, 'txt_r_mean': 30.72, 'img_r1': 10.748130672957735, 'img_r5': 27.198208644887842, 'img_r10': 38.02631052820985, 'img_r_mean': 25.324216615351805, 'r_mean': 28.0221083076759}
zeroshot: {'zeroshot_top1': 26.7, 'zeroshot_top3': 40.006, 'zeroshot_top5': 45.718, 'zeroshot_top10': 52.65}
Training time 0:40:05


## Results

In [10]:
import pandas as pd
Results= pd.DataFrame()
Results['Method']=["CLIP + AdamW", "CLIP + Novograd", "isogclr_new + AdamP", "isogclr_new + RAdam"]
Results['MSCOCO TR@1']=[12.22, 9.84, 13.96, 13.66]
Results['MSCOCO IR@1']=[9.07, 6.89, 10.58, 10.75]
Results['ImageNet ACC@1']=[21.36, 14.17, 26.08, 26.7]
Results['Average'] = Results[['MSCOCO TR@1', 'MSCOCO IR@1', 'ImageNet ACC@1']].mean(axis=1)
Results

,Method,MSCOCO TR@1,MSCOCO IR@1,ImageNet ACC@1,Average
0,CLIP + AdamW,12.22,9.07,21.36,14.216667
1,CLIP + Novograd,9.84,6.89,14.17,10.300000
2,isogclr_new + AdamP,13.96,10.58,26.08,16.873333
3,isogclr_new + RAdam,13.66,10.75,26.70,17.036667


### Benchmarks

The following results are recall at 1 results on the provided MSCOCO and ImageNet datasets. The first row of results are from the model trained using the CLIP loss, and the second row of results are from the model trained using the SogCLR loss. All results are based on a batch size of 128 for 30-epoch pretraining. IR@1 denotes the recall at 1 of image retrieval on MSCOCO, TR@1 denotes the recall at 1 of text retrieval on MSCOCO, and ACC@1 denotes the top 1 accuracy on ImageNet. Average denotes the average of the three metrics.

| Method | MSCOCO TR@1 | MSCOCO IR@1 | ImageNet ACC@1 | Average |
|:----------:|:--------:|:--------:|:--------:|:--------:|
| CLIP | 12.0 | 9.32 | 21.35 | 14.22 |
| SogCLR |  14.38  |  10.73  | 24.54 | 16.55 |

## Novelty

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/noveltycl_and_RAdam \
    --init_model \
    --use_amp \
    --ita_type noveltycl \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --opt RAdam

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python ./iSogCLR/bimodal_exps/clip.py \
    --data_path ./datasets \
    --ann_path ./clip_train \
    --train_file cc3m_train_subset.json \
    --train_image_root cc3m_subset_100k \
    --output_dir output/noveltycl_and_RAdam \
    --init_model \
    --use_amp \
    --ita_type noveltycl \
    --tau_init 0.01 \
    --sogclr_gamma 0.8 \
    --eta_init 0.03 --sched cosine \
    --no-distributed \
    --epochs 30 \
    --evaluate --checkpoint './output/noveltycl_and_RAdam/checkpoint_30.pth' \
    --zs_dataset imagenet --zs_datafolder ./datasets/imagenet/val